In [2]:
import os
import json
from dotenv import load_dotenv
from pathlib import Path
import random
import pandas as pd

dotenv_path = Path("../../.env")
if dotenv_path.exists():
    print("Loaded .env file!")
    load_dotenv(str(dotenv_path))


# data = json.load(open(Path(os.environ["PROJECT_HOME_DIR"]) / "data/mpware_mixtral8x7b_v1.1-no-i-username.json"))

In [2]:
tokens = []
labels = []
document = []
token_text = []

for d in data:
    tokens.extend(list(range(len(d["tokens"]))))
    token_text.extend(d["tokens"])
    labels.extend(d["labels"])
    document.extend([d["document"]] * len(d["tokens"]))

df = pd.DataFrame({"document": document, "token": tokens, "token_text": token_text, "label": labels})

df.to_parquet(Path(os.environ["PROJECT_HOME_DIR"]) / "data/mpware.pq")

In [3]:
gt_df = pd.read_parquet(Path(os.environ["PROJECT_HOME_DIR"]) / "data/mpware.pq")

d3l_preds = pd.read_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-floral-bird-887-mpware/preds.pq")
d2xl_preds = pd.read_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/d2xl-frosty-grass-890-mpware/preds.pq")

In [4]:
from piidd.training.utils import pii_fbeta_score_v2

from piidd.processing.post import manual_v2


d3l_pdf = manual_v2(d3l_preds)


pii_fbeta_score_v2(d3l_pdf[d3l_pdf.label != "O"], gt_df[gt_df.label != "O"], ".", beta=5, save_csv=True)

/drive2/kaggle/pii-dd/piidd/processing/post.py:594: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pred_df["token_text"].str.contains("(http)|(www)|(://)"),
/drive2/kaggle/pii-dd/piidd/processing/post.py:669: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  url_likely = pred_df["token_text"].str.contains("(http)|(www)|(://)")


0.8608862094327752

In [6]:
d2xl_pdf = manual_v2(d2xl_preds)


pii_fbeta_score_v2(d2xl_pdf[d2xl_pdf.label != "O"], gt_df[gt_df.label != "O"], ".", beta=5, save_csv=True)

/drive2/kaggle/pii-dd/piidd/processing/post.py:590: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pred_df["token_text"].str.contains("(http)|(www)|(://)"),
/drive2/kaggle/pii-dd/piidd/processing/post.py:665: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  url_likely = pred_df["token_text"].str.contains("(http)|(www)|(://)")


0.8450168819209973

In [5]:
d3_results = pd.read_parquet("/drive2/kaggle/pii-dd/notebooks/20240422133102_pii_fbeta.pq")
d2_results = pd.read_parquet("/drive2/kaggle/pii-dd/notebooks/20240422133128_pii_fbeta.pq")

In [6]:
d3 = d3_results[["token", "document", "cm", "label_gt"]].merge(d3l_preds, on=["token", "document"])

d3

,token,document,cm,label_gt,token_text,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O,row_id
0,879,003e6c9a31364c314417b43fdf8950bc,TP,B-NAME_STUDENT,Kayla,0.000000,1.221375e-05,0.999335,1.391393e-05,4.952821e-05,0.000012,0.000000,4.010483e-06,0.000009,2.129301e-06,6.014009e-06,0.000536,93412
1,880,003e6c9a31364c314417b43fdf8950bc,TP,I-NAME_STUDENT,Todd,0.000000,4.358994e-06,0.000012,3.102735e-06,1.884364e-06,0.000008,0.000000,3.415441e-06,0.999854,8.433080e-06,1.430840e-05,0.000078,93413
2,883,003e6c9a31364c314417b43fdf8950bc,TP,B-EMAIL,kchurch@outlook.com,0.999922,3.293748e-06,0.000000,1.027072e-05,5.614376e-06,0.000011,0.000007,5.855184e-06,0.000000,4.568027e-06,4.578062e-06,0.000085,93416
3,886,003e6c9a31364c314417b43fdf8950bc,TP,B-USERNAME,lmaldonado,0.000000,8.657674e-06,0.000000,2.868269e-06,4.834298e-06,0.000010,0.999941,3.338314e-06,0.000000,1.964137e-06,3.732347e-06,0.000032,93419
4,889,003e6c9a31364c314417b43fdf8950bc,TP,B-ID_NUM,GB38GHTL21117908075957,0.000000,9.997303e-01,0.000000,1.837220e-04,2.422975e-05,0.000022,0.000000,2.597415e-05,0.000000,5.878079e-06,5.440582e-06,0.000098,93422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27127,136,67d7d14e3a7aecf177724ea8e5a15298,FN,I-NAME_STUDENT,Carlson,0.000000,6.109825e-06,0.000142,2.276279e-06,2.844320e-06,0.000012,0.000000,2.742188e-06,0.000142,4.246448e-06,4.884952e-06,0.999671,1819496
27128,405,67d7d14e3a7aecf177724ea8e5a15298,FN,B-NAME_STUDENT,John,0.000000,3.338532e-06,0.000352,2.212930e-06,2.979637e-06,0.000010,0.000000,1.378500e-06,0.000010,2.026680e-06,1.971852e-06,0.999907,1819765
27129,406,67d7d14e3a7aecf177724ea8e5a15298,FN,I-NAME_STUDENT,Carlson,0.000000,5.056467e-06,0.000126,1.734299e-06,3.009066e-06,0.000011,0.000000,2.914304e-06,0.000165,4.447553e-06,5.146293e-06,0.999926,1819766
27130,604,67d7d14e3a7aecf177724ea8e5a15298,FN,B-NAME_STUDENT,John,0.000000,8.896527e-07,0.000079,5.297869e-07,7.610685e-07,0.000002,0.000000,4.701084e-07,0.000003,4.430289e-07,6.224943e-07,0.999911,1819964


In [35]:
tds = pd.read_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-floral-bird-887-mpware/tds.pq")
ds = pd.read_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-floral-bird-887-mpware/ds.pq")

In [66]:
ds[ds["document"]=="dc1400cfe74846fd8b16acf6921ae1bf"]["token_map"].item()[5080:]

array([900, 900, 901, 902, 902, 902, 902, 902, 902, 902, 902, 903,  -1,
       904, 904, 904, 904, 905, 906, 906,  -1, 907, 907, 907, 908,  -1,
       909, 910, 911, 911, 911, 911, 911, 912, 913, 913, 913, 913, 913,
        -1, 914, 914, 914, 915,  -1, 916, 916, 916, 916, 916, 916, 916,
       916, 916, 916, 917, 918, 918, 918,  -1, 919, 919, 919, 919, 919,
       919, 919, 919, 920,  -1, 921, 921, 921, 921, 921, 921, 921, 921,
       921, 921, 921, 921, 921, 921, 921, 921, 921, 921, 921, 921, 921,
       921, 921, 921, 921, 921, 921, 921, 921, 922, 923, 923, 923, 923,
       923, 923,  -1, 924, 924, 924, 924, 924, 924, 924, 925,  -1, 926,
       926, 926, 926,  -1, 927, 927, 927, 927, 927,  -1, 928, 928, 928,
       928, 928, 928, 928,  -1, 929, 929, 929, 929, 929, 929, 929, 929,
       929, 930,  -1, 931, 931,  -1, 932, 932, 932, 932, 932])

In [78]:
ds

,full_text,document,tokens,trailing_whitespace,provided_labels,idx,token_map
0,Storytelling Challenge: Crafting Compelling Na...,2cd5fb9799fe06349528c33423a0f595,"[Storytelling, Challenge, :, Crafting, Compell...","[True, False, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1,..."
1,Storytelling Challenge: The Power of Narrative...,3cb7ffeb0ef2f45672e53c38b29d610c,"[Storytelling, Challenge, :, The, Power, of, N...","[True, False, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1,..."
2,Storytelling Challenge: The Power of Narrative...,c172f453e4643ae9c9bc9d8426128217,"[Storytelling, Challenge, :, The, Power, of, N...","[True, False, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1,..."
3,Storytelling Challenge: Crafting Compelling Na...,b9edf42553f58bbffa457381a3799770,"[Storytelling, Challenge, :, Crafting, Compell...","[True, False, True, True, True, True, True, Fa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1,..."
4,Storytelling Challenge: The Power of Narrative...,232dd2bd7be9d3a3ea93cab6e17fcbfa,"[Storytelling, Challenge, :, The, Power, of, N...","[True, False, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 1,..."
...,...,...,...,...,...,...,...
2687,Mind Mapping Tool for Idea Generation and Prob...,72efe6ee9844b8c802098af0585397bd,"[Mind, Mapping, Tool, for, Idea, Generation, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2687,"[0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, -1, 2, 2..."
2688,Mind Mapping Tool for Idea Generation and Prob...,d2ceb11151ea660a3d20238e36c169f1,"[Mind, Mapping, Tool, for, Idea, Generation, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2688,"[0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, -1, 2, 2..."
2689,Mind Mapping Tools for Idea Generation and Pro...,21bce330d38baa5be1a7018e1bf82fe2,"[Mind, Mapping, Tools, for, Idea, Generation, ...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2689,"[0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, -1, 2, 2..."
2690,Mind Mapping Tool for Idea Generation and Prob...,0d876118e85ef8a4720a7f12783f7dc3,"[Mind, Mapping, Tool, for, Idea, Generation, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2690,"[0, 0, 0, 0, -1, 1, 1, 1, 1, 1, 1, 1, -1, 2, 2..."


In [41]:
tds[tds["idx"]==196].index

Int64Index([294, 295, 5725], dtype='int64')

In [43]:
p.shape, tds.shape

((6332, 512, 12), (6332, 8))

In [52]:
x = tds[tds["idx"]==196]["offset_mapping"].tolist()[-1]

In [60]:
for i, y in enumerate(x):
    if y[0] == 5082:
        print(i)

168


In [79]:
p[294]

array([[ 0.21029918,  0.27427247, -0.02833996, ..., -0.01574648,
         0.76723015,  0.9383412 ],
       [-1.6254758 , -1.8160512 , -0.19580494, ..., -2.0697663 ,
        -1.3013632 , 12.209717  ],
       [-1.4824657 , -1.9458457 , -0.30326828, ..., -2.456257  ,
        -2.3281772 , 13.73204   ],
       ...,
       [-1.4403629 , -1.8520238 , -0.40130654, ..., -2.4144588 ,
        -1.5580132 , 14.259897  ],
       [-1.4369556 , -0.96288705, -0.01734452, ..., -2.1267166 ,
        -1.3636262 , 11.934739  ],
       [-0.99268556, -0.87089896,  0.96285826, ..., -1.8398318 ,
        -1.1805758 ,  7.8538294 ]], dtype=float32)

In [76]:
p[[294, 295, 5725]][-1][168:180].a

array([[-5.000e-01, -8.900e-01, -1.200e+00, -1.710e+00, -2.580e+00,
        -1.150e+00, -1.610e+00, -1.910e+00, -1.440e+00, -2.430e+00,
        -1.070e+00,  1.337e+01],
       [-1.190e+00, -1.580e+00, -7.100e-01, -1.880e+00, -2.300e+00,
        -8.100e-01, -1.850e+00, -2.210e+00, -1.380e+00, -2.590e+00,
        -1.740e+00,  1.341e+01],
       [-1.430e+00, -7.600e-01, -8.400e-01, -1.490e+00, -2.010e+00,
        -5.600e-01, -1.240e+00, -2.480e+00, -1.680e+00, -3.230e+00,
        -1.430e+00,  1.359e+01],
       [-6.200e-01,  4.100e-01,  1.000e-01, -2.100e+00, -1.900e+00,
        -9.200e-01,  1.047e+01, -1.860e+00, -1.790e+00, -3.000e+00,
        -2.130e+00,  7.700e-01],
       [-3.400e-01, -2.200e-01,  1.000e-02, -2.250e+00, -1.870e+00,
        -8.600e-01,  1.068e+01, -1.680e+00, -1.180e+00, -2.600e+00,
        -2.080e+00,  5.200e-01],
       [-1.630e+00, -3.500e-01, -9.900e-01, -1.880e+00, -2.510e+00,
        -1.690e+00, -4.900e-01, -1.530e+00, -1.220e+00, -2.910e+00,
        -9.500e-01,

In [63]:
{0: 'B-EMAIL',
 1: 'B-ID_NUM',
 2: 'B-NAME_STUDENT',
 3: 'B-PHONE_NUM',
 4: 'B-STREET_ADDRESS',
 5: 'B-URL_PERSONAL',
 6: 'B-USERNAME',
 7: 'I-ID_NUM',
 8: 'I-NAME_STUDENT',
 9: 'I-PHONE_NUM',
 10: 'I-STREET_ADDRESS',
 11: 'O'}

NameError: name 'model' is not defined

In [82]:
d3l_preds[d3l_preds[['B-EMAIL', 'B-ID_NUM',
       'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL',
       'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM',
       'I-STREET_ADDRESS', 'O']].sum(axis=1) < 0.8].tail(20)

,token,document,token_text,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O,row_id


In [84]:
d3l_preds[d3l_preds.document == "dc1400cfe74846fd8b16acf6921ae1bf"].tail(30)

,token,document,token_text,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O,row_id
162097,903,dc1400cfe74846fd8b16acf6921ae1bf,:,0.0,5.842591e-07,0.000000e+00,2.816783e-07,1.674964e-07,0.000000e+00,0.0,1.045189e-07,0.000000e+00,4.932633e-08,2.985316e-07,0.999996,162097
162098,904,dc1400cfe74846fd8b16acf6921ae1bf,jali,0.0,4.296967e-05,0.000000e+00,3.480980e-06,4.268697e-06,0.000000e+00,0.0,4.448247e-06,0.000000e+00,1.720317e-06,3.401105e-06,0.000062,162098
162099,905,dc1400cfe74846fd8b16acf6921ae1bf,\n,0.0,0.000000e+00,0.000000e+00,0.000000e+00,2.111929e-07,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.011566e-06,0.999992,162099
162100,906,dc1400cfe74846fd8b16acf6921ae1bf,ID,0.0,5.929930e-07,0.000000e+00,2.266150e-07,1.738507e-07,0.000000e+00,0.0,1.424786e-07,0.000000e+00,8.717701e-08,3.048182e-07,0.999996,162100
162101,907,dc1400cfe74846fd8b16acf6921ae1bf,Num,0.0,7.685645e-07,4.742431e-07,1.783699e-07,1.453792e-07,0.000000e+00,0.0,2.314577e-07,2.567702e-07,8.010673e-08,2.872578e-07,0.999997,162101
162102,908,dc1400cfe74846fd8b16acf6921ae1bf,:,0.0,2.339157e-06,0.000000e+00,3.070227e-07,1.946653e-07,0.000000e+00,0.0,2.763577e-07,0.000000e+00,4.693555e-08,2.840850e-07,0.999995,162102
162103,909,dc1400cfe74846fd8b16acf6921ae1bf,9,0.0,9.995048e-01,0.000000e+00,8.674047e-05,5.102434e-05,0.000000e+00,0.0,5.715224e-05,0.000000e+00,3.565691e-06,6.664308e-06,0.000218,162103
162104,910,dc1400cfe74846fd8b16acf6921ae1bf,-,0.0,8.367636e-01,0.000000e+00,3.981157e-05,8.953829e-05,0.000000e+00,0.0,1.611118e-01,0.000000e+00,3.280122e-04,3.401905e-04,0.000528,162104
162105,911,dc1400cfe74846fd8b16acf6921ae1bf,G8554,0.0,7.218186e-01,0.000000e+00,4.054143e-05,4.309510e-05,5.768973e-05,0.0,6.468816e-01,0.000000e+00,2.055325e-04,1.956314e-04,0.000195,162105
162106,912,dc1400cfe74846fd8b16acf6921ae1bf,\n,0.0,0.000000e+00,0.000000e+00,0.000000e+00,2.997719e-07,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.002680e-06,0.999990,162106


In [12]:
d3[d3.cm == "FN"].sample(100).head(20)

,token,document,cm,label_gt,token_text,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O,row_id
25774,104,bc09694ff0ddda6e7eb3cd9398305e06,FN,B-USERNAME,AracelyMuller99,0.0,0.000062,0.000000,0.000010,0.000014,0.000038,0.0,0.000011,0.000000,0.000004,0.000011,0.000089,485794
26246,546,38d08d46cccf15f5ac9a099826322819,FN,I-NAME_STUDENT,Koch,0.0,0.000019,0.001723,0.000012,0.000018,0.000079,0.0,0.000017,0.005720,0.000021,0.000056,0.992267,1221501
26398,14,931f1bd1af569a62a5cac40c8b2dbe97,FN,B-NAME_STUDENT,Danny,0.0,0.000102,0.113467,0.000084,0.000095,0.000211,0.0,0.000062,0.000447,0.000045,0.000077,0.885223,1314130
25453,770,6bd5d70edf78970dfa90c63a988e913f,FN,I-NAME_STUDENT,-,0.0,0.000007,0.000000,0.000004,0.000006,0.000000,0.0,0.000004,0.000000,0.000048,0.000051,0.000063,84531
26201,14,e45a75b1642c06b8b21ed570eb2cbbf4,FN,B-NAME_STUDENT,Samantha,0.0,0.000047,0.069383,0.000058,0.000062,0.000181,0.0,0.000047,0.000634,0.000026,0.000062,0.929339,1196777
25808,505,5be433f5fa58936cf82526cfa869313d,FN,I-NAME_STUDENT,Evans,0.0,0.000349,0.024392,0.000096,0.000051,0.000180,0.0,0.000137,0.112857,0.000286,0.000433,0.860833,557138
25550,295,5b66a644aac9b7dc0457bed9acd33c29,FN,I-NAME_STUDENT,-,0.0,0.002165,0.000000,0.001288,0.000251,0.000000,0.0,0.000958,0.000000,0.007754,0.004463,0.370313,186460
27099,651,f9d4312d9d657f2ec00dd585ae34f2a6,FN,B-NAME_STUDENT,Jody,0.0,0.000006,0.002378,0.000005,0.000006,0.000007,0.0,0.000002,0.000022,0.000002,0.000005,0.997555,1814730
26597,442,5ca3a80b4c40b2f4b5d5c335a97e99fa,FN,I-NAME_STUDENT,Harrison,0.0,0.000017,0.000382,0.000009,0.000007,0.000025,0.0,0.000010,0.000844,0.000021,0.000018,0.999236,1480150
26171,17,39a6a6c5bb2b4004c2ecae738245b524,FN,I-NAME_STUDENT,Taylor,0.0,0.000007,0.000288,0.000002,0.000003,0.000043,0.0,0.000007,0.000742,0.000006,0.000011,0.998879,1167713


In [29]:
d = [d for d in data if d["document"] == "dc1400cfe74846fd8b16acf6921ae1bf"]

In [46]:
d[0]["full_text"].index("Username: jali")

5083